In [21]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [22]:
#get all apps > 10 m tvl
all_api = 'https://api.llama.fi/protocols'
res = pd.DataFrame( r.get(all_api, headers=header).json() )
res = res[res['tvl'] > 25_000_000] ##greater than 10mil
print(len(res))
# print(res.columns)

235


In [23]:

protocols = res[['slug','chainTvls']]
re = res['chainTvls']
# r[1].keys()
protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )
protocols

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_5334/1800157470.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  protocols['chainTvls'] = protocols['chainTvls'].apply(lambda x: list(x.keys()) )


,slug,chainTvls
0,makerdao,[Ethereum]
1,polygon-bridge-&-staking,[Polygon]
2,lido,"[Ethereum, Moonriver, Solana, Terra, Moonbeam]"
3,uniswap,"[Optimism, Ethereum, Polygon, Celo, Arbitrum]"
4,curve,"[Optimism, Harmony, Moonbeam, Ethereum, Aurora..."
...,...,...
230,aldrin,[Solana]
231,sharedstake,"[Ethereum, pool2, Ethereum-pool2]"
232,solo-top,"[Binance, OKExChain, Heco, Polygon]"
233,rook,[Ethereum]


In [24]:

# api_str = 'https://api.llama.fi/protocol/'
# ad = pd.DataFrame( r.get(api_str).json()['chainTvls'] ).T[['tokens']]
# ad

In [25]:
#get by app
api_str = 'https://api.llama.fi/protocol/'
# print(protocols)
prod = []
for index,proto in protocols.iterrows():
#     print(proto)
    prot = proto['slug']
    chains = proto['chainTvls']
    apic = api_str + prot
    time.sleep(0.1)
    try:
        prot_req = r.get(apic, headers=header).json()['chainTvls']
        print(apic)
        for ch in chains:
            ad = pd.json_normalize( prot_req[ch]['tokens'] )
            ad_usd = pd.json_normalize( prot_req[ch]['tokensInUsd'] )
#             ad = ad.merge(how='left')
            if not ad.empty:
                ad = pd.melt(ad,id_vars = ['date'])
                ad = ad.rename(columns={'variable':'token','value':'token_value'})
                ad_usd = pd.melt(ad_usd,id_vars = ['date'])
                ad_usd = ad_usd.rename(columns={'variable':'token','value':'usd_value'})
                ad = ad.merge(ad_usd,on=['date','token'])
                
                ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
                ad['token'] = ad['token'].str.replace('tokens.','')
                ad['protocol'] = prot
                ad['chain'] = ch
        #         ad['start_date'] = pd.to_datetime(prot[1])
                ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
                prod.append(ad)
    except:
        print('err')

df_df = pd.concat(prod)
df_df

https://api.llama.fi/protocol/makerdao


/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_5334/1041932425.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  ad['token'] = ad['token'].str.replace('tokens.','')


https://api.llama.fi/protocol/polygon-bridge-&-staking
https://api.llama.fi/protocol/lido
https://api.llama.fi/protocol/uniswap
https://api.llama.fi/protocol/curve
https://api.llama.fi/protocol/wbtc
https://api.llama.fi/protocol/aave-v2
https://api.llama.fi/protocol/convex-finance
https://api.llama.fi/protocol/justlend
https://api.llama.fi/protocol/pancakeswap
https://api.llama.fi/protocol/compound
https://api.llama.fi/protocol/instadapp
https://api.llama.fi/protocol/multichain
https://api.llama.fi/protocol/arrakis-finance
https://api.llama.fi/protocol/aave-v3
https://api.llama.fi/protocol/balancer
https://api.llama.fi/protocol/justcryptos
https://api.llama.fi/protocol/frax
https://api.llama.fi/protocol/juststables
https://api.llama.fi/protocol/hbtc
https://api.llama.fi/protocol/sunswap
https://api.llama.fi/protocol/sushiswap
https://api.llama.fi/protocol/venus
https://api.llama.fi/protocol/vvs-finance
https://api.llama.fi/protocol/synthetix
https://api.llama.fi/protocol/yearn-finance


https://api.llama.fi/protocol/spool-protocol
https://api.llama.fi/protocol/hector-network
https://api.llama.fi/protocol/hashflow
https://api.llama.fi/protocol/harvest-finance
https://api.llama.fi/protocol/rex-staking
https://api.llama.fi/protocol/xdao
https://api.llama.fi/protocol/solarbeam
https://api.llama.fi/protocol/liqee
https://api.llama.fi/protocol/stafi
https://api.llama.fi/protocol/nftx
https://api.llama.fi/protocol/thetaswap
https://api.llama.fi/protocol/hubble
https://api.llama.fi/protocol/dopex
https://api.llama.fi/protocol/maple
https://api.llama.fi/protocol/reserve
https://api.llama.fi/protocol/filda
https://api.llama.fi/protocol/ooki
https://api.llama.fi/protocol/unslashed
https://api.llama.fi/protocol/acryptos
https://api.llama.fi/protocol/sovryn
https://api.llama.fi/protocol/iziswap
https://api.llama.fi/protocol/tarot
https://api.llama.fi/protocol/aldrin
https://api.llama.fi/protocol/sharedstake
https://api.llama.fi/protocol/solo-top
https://api.llama.fi/protocol/rook


,date,token,token_value,usd_value,protocol,chain
0,2022-05-09 00:00:00,MATIC,61211089.17387,5.060529e+07,makerdao,Ethereum
1,2022-05-10 00:00:00,MATIC,61211089.17387,5.437283e+07,makerdao,Ethereum
2,2022-05-11 00:00:00,MATIC,131231089.17387,8.719886e+07,makerdao,Ethereum
3,2022-05-12 00:00:00,MATIC,131264317.97387,7.932946e+07,makerdao,Ethereum
4,2022-05-13 00:00:00,MATIC,131264317.97387,9.167802e+07,makerdao,Ethereum
...,...,...,...,...,...,...
1279,2022-08-03 00:00:00,FTM,NaN,NaN,connext,Fantom
1280,2022-08-04 00:00:00,FTM,NaN,NaN,connext,Fantom
1281,2022-08-05 00:00:00,FTM,NaN,NaN,connext,Fantom
1282,2022-08-06 00:00:00,FTM,NaN,NaN,connext,Fantom


In [129]:
# df_df[df_df['protocol']=='perpetual-protocol']
df_df = df_df[df_df['date'].dt.date >= date.today()-timedelta(days=30) ]
df_df

,date,token,token_value,usd_value,protocol,chain
60,2022-07-08 00:00:00,MATIC,130919948.07467,7.830859e+07,makerdao,Ethereum
61,2022-07-09 00:00:00,MATIC,130919948.07467,7.714995e+07,makerdao,Ethereum
62,2022-07-10 00:00:00,MATIC,130919948.07467,7.372076e+07,makerdao,Ethereum
63,2022-07-11 00:00:00,MATIC,130919948.07467,7.387944e+07,makerdao,Ethereum
64,2022-07-12 00:00:00,MATIC,130919948.07467,7.100129e+07,makerdao,Ethereum
...,...,...,...,...,...,...
1279,2022-08-03 00:00:00,FTM,NaN,NaN,connext,Fantom
1280,2022-08-04 00:00:00,FTM,NaN,NaN,connext,Fantom
1281,2022-08-05 00:00:00,FTM,NaN,NaN,connext,Fantom
1282,2022-08-06 00:00:00,FTM,NaN,NaN,connext,Fantom


In [130]:
#defillama api feedback - only token symbols come through, makes it hard to map w/o doing it manually
coingecko_token_map = [
    ['LINK','chainlink']
    ,['USDT','tether']
    ,['USDC','usd-coin']
    ,['WETH','weth']
    ,['SUSD','nusd']
    ,['DAI','dai']
    ,['AAVE','aave']
    ,['WBTC','wrapped-bitcoin']
    ,['SNX','havven']
    ,['OP','optimism']
    ,['SETH','seth']
    ,['FXS','frax-share']
    ,['THALES','thales']
    ,['FRAX','frax']
    ,['ALUSD','alchemix-usd']
    ,['PERP','perpetual-protocol']
    ,['LUSD','liquity-usd']
    ,['LYRA','lyra-finance']
    ,['HND','hundred-finance']
    ,['BITANT','bitant']
    ,['UNI','uniswap']
    ,['SLINK','slink']
    ,['VELO','velodrome-finance']
    ,['DOLA','dola-usd']
    ,['CRV','curve-dao-token']
    ,['SBTC','sbtc']
    ,['KROM','kromatika']
    ,['DF','dforce-token']
    ,['STG','stargate-finance']
    ,['AELIN','aelin']
    ,['RAI','rai']
    ,['RETH','rocket-pool-eth']
    ,['MATIC','']
    ,['RENBTC','']
    ,['COMP','']
    ,['BUSD','']
    ,['SUSHI','']
    ,['WBNB','']
    ,['MIM','']
    ,['UST','']
    ,['TUSD','']
    ,['YFI','']
    ,['WAVAX','']
    ,['FTM','']
    ,['WMATIC','']
    ,['ETH','']
    ,['tether','']
    ,['usd-coin','']
    ,['GOHM','']
    ,['UST','']
    ,['USTC','']
    ,['CAKE','']
    ,['BTCB','']
    ,['FEI','']
    ,['DPI','']
    ,['bitcoin','']
    ,['MKR','']
    ,['SPELL','']
    ,['BAT','']
    ,['SHIB','']
    ,['BAL','']
    ,['MIMATIC','']
    ,['1INCH','']
    ,['EURS','']
    ,['ZRX','']
    ,['MANA','']
    ,['USDP','']
    ,['WOO','']
    
]
cg_token_list = [i[0] for i in coingecko_token_map]

In [131]:
# DISTINCT TOKENS

token_list = df_df.groupby(['token']).count()
token_list = token_list[token_list['token_value']>=1000]
token_list.sort_values(by='token_value',inplace=True, ascending=False)
token_list.reset_index(inplace=True)
missing_token_list = token_list[~token_list['token'].isin(cg_token_list)]
missing_token_list
# missing_token_list

,token,date,token_value,usd_value,protocol,chain


In [132]:

# cg_pds = []
# for t in coingecko_token_map:
#     cg_api = 'https://api.coingecko.com/api/v3/coins/'\
#             + t[1] + '/market_chart?vs_currency=usd&days=max&interval=daily'
#     pr = pd.DataFrame( r.get(cg_api, headers=header).json()['prices'] )
#     pr['token'] = t[0]
#     pr['cg_slug'] = t[1]
#     pr = pr.rename(columns={0:'date',1:'price_usd'})
#     pr['date'] = pd.to_datetime(pr['date'], unit ='ms') #convert to days
#     cg_pds.append(pr)

# cg_df = pd.concat(cg_pds)
# # cg_df

In [133]:
# data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df = df_df.copy()
data_df['token_value'] = data_df['token_value'].replace(0, np.nan)
data_df['price_usd'] = data_df['usd_value']/data_df['token_value']

# print(len(data_df))
data_df.sort_values(by='date',inplace=True)
# print(len(data_df))
data_df['last_token_value'] = data_df.groupby(['token','protocol','chain'])['token_value'].shift(1)
# data_df['net_token_flow'] = data_df.groupby(['token','protocol','chain'])['token_value'].apply(lambda x: x- x.shift(1))
data_df['net_token_flow'] = data_df['token_value'] - data_df['last_token_value']

# print(len(data_df))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']


data_df = data_df[data_df['token'] != 'RVRS'] #Harmony error
data_df = data_df[data_df['token'] != 'usn'] #Harmony error
data_df = data_df[data_df['protocol'] != 'shibaswap'] #Harmony error
data_df = data_df[abs(data_df['net_dollar_flow']) < 50_000_000_000] #50 bil error bar

# display(data_df[
# #         (data_df['protocol'] == 'sushiswap') & (data_df['chain'] == 'Harmony')
#         (data_df['net_dollar_flow'].notna()) & (data_df['net_dollar_flow'] != 0)
#         ].sort_values('net_dollar_flow'))


data_df[(data_df['protocol'] == 'justlend')]


,date,token,token_value,usd_value,protocol,chain,price_usd,last_token_value,net_token_flow,net_dollar_flow
251,2022-07-09 00:00:00,USDT,3281722137.31244,3.285004e+09,justlend,Tron,1.001,3305625134.46957,-23902997.15713,-23926900.154287
252,2022-07-10 00:00:00,USDT,3254898364.76114,3.253717e+09,justlend,Tron,0.999637,3281722137.31244,-26823772.5513,-26814035.521864
253,2022-07-11 00:00:00,USDT,3203502111.65755,3.203502e+09,justlend,Tron,1.0,3254898364.76114,-51396253.10359,-51396253.10359
254,2022-07-12 00:00:00,USDT,3171435068.00034,3.165359e+09,justlend,Tron,0.998084,3203502111.65755,-32067043.65721,-32005603.201563
255,2022-07-13 00:00:00,USDT,3210861272.26685,3.210861e+09,justlend,Tron,1.0,3171435068.00034,39426204.26651,39426204.26651
256,2022-07-14 00:00:00,USDT,3203263358.84629,3.203263e+09,justlend,Tron,1.0,3210861272.26685,-7597913.42056,-7597913.42056
257,2022-07-15 00:00:00,USDT,3240669233.95191,3.243910e+09,justlend,Tron,1.001,3203263358.84629,37405875.10562,37443280.980726
258,2022-07-16 00:00:00,USDT,3272666774.08635,3.272667e+09,justlend,Tron,1.0,3240669233.95191,31997540.13444,31997540.13444
259,2022-07-17 00:00:00,USDT,3257700199.3415,3.257700e+09,justlend,Tron,1.0,3272666774.08635,-14966574.74485,-14966574.74485
260,2022-07-18 00:00:00,USDT,3330212002.84841,3.340203e+09,justlend,Tron,1.003,3257700199.3415,72511803.50691,72729338.917431


In [134]:
# data_df[data_df['protocol']=='perpetual-protocol'].sort_values(by='date')

# segment_cols = ['date','protocol','chain','token']

In [135]:
netdf_df = data_df[['date','protocol','chain','net_dollar_flow','token']]
netdf_df = netdf_df.fillna(0)
# display(netdf_df)
netdf_df = netdf_df.groupby(['date','protocol','chain','token']).sum(['net_dollar_flow'])

netdf_df['cumul_net_dollar_flow'] = netdf_df.groupby(['protocol','chain','token']).cumsum()
netdf_df.reset_index(inplace=True)

netdf_df

,date,protocol,chain,token,net_dollar_flow,cumul_net_dollar_flow
0,2022-07-09 00:00:00,aave-v1,Ethereum,AAVE,0.000000,0.000000
1,2022-07-09 00:00:00,aave-v1,Ethereum,BAT,0.000000,0.000000
2,2022-07-09 00:00:00,aave-v1,Ethereum,BUSD,0.000000,0.000000
3,2022-07-09 00:00:00,aave-v1,Ethereum,DAI,-174.792948,-174.792948
4,2022-07-09 00:00:00,aave-v1,Ethereum,ENJ,0.000000,0.000000
...,...,...,...,...,...,...
260609,2022-08-06 02:59:01,yield-yak,Avalanche-pool2,YAK,96.461263,-3827.567141
260610,2022-08-06 02:59:01,yield-yak,Avalanche-staking,YAK,9.516525,103642.770832
260611,2022-08-06 02:59:01,yield-yak,pool2,WAVAX,-114.148551,3525.306742
260612,2022-08-06 02:59:01,yield-yak,pool2,YAK,96.461263,-3827.567141


In [138]:
summary_df = netdf_df[(netdf_df['date'].dt.date == date.today()-timedelta(days=1)) &\
                        (~netdf_df['chain'].str.contains('borrowed')) &\
                        (~netdf_df['chain'].str.contains('staking'))  
                        ]
summary_df = summary_df.sort_values(by='cumul_net_dollar_flow',ascending=False)
summary_df['direction'] = np.where(summary_df['cumul_net_dollar_flow']>=0,1,0)
summary_df['abs_cumul_net_dollar_flow'] = abs(summary_df['cumul_net_dollar_flow'])
# display(summary_df)
fig = px.treemap(summary_df[summary_df['abs_cumul_net_dollar_flow'] !=0], \
                 path=[px.Constant("all"), 'chain', 'protocol','token'], \
#                  path=[px.Constant("all"), 'token', 'chain', 'protocol'], \
                 values='abs_cumul_net_dollar_flow', color='direction'
                ,color_discrete_map={'(?)':'lightgrey', '0':'red', '1':'green'})
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.update_layout(tickprefix = '$')
fig.show()

In [137]:
# netdf_df_plot = netdf_df[netdf_df['chain']=='Ethereum']

# fig = px.line(netdf_df_plot, x="date", y="net_dollar_flow", color="protocol", \
#              title="Daily Net Dollar Flow since Program Announcement",\
#             labels={
#                      "date": "Day",
#                      "net_dollar_flow": "Net Dollar Flow (N$F)"
#                  }
#             )
# fig.update_layout(
#     legend_title="App Name"
# )
# fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

# # cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
# #              title="Cumulative Dollar Flow since Program Announcement",\
# #                    labels={
# #                      "date": "Day",
# #                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
# #                  }
# #             ,areamode='group')
# # cumul_fig.update_layout(yaxis_tickprefix = '$')
# # cumul_fig.show()


# cumul_fig = go.Figure()
# proto_names = netdf_df_plot['protocol'].drop_duplicates()
# print(proto_names)
# for p in proto_names:
#     cumul_fig.add_trace(go.Scatter(x=netdf_df_plot[netdf_df_plot['protocol'] == p]['date'] \
#                                    , y=netdf_df_plot[netdf_df_plot['protocol'] == p]['cumul_net_dollar_flow'] \
#                                     ,name = p\
#                                   ,fill='tozeroy')) # fill down to xaxis

# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.update_layout(
#     title="Cumulative Dollar Flow since Program Announcement",
#     xaxis_title="Day",
#     yaxis_title="Cumulative Net Dollar Flow (N$F)",
#     legend_title="App Name",
# #     color_discrete_map=px.colors.qualitative.G10
# )
# cumul_fig.show()